In [7]:
import keras
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import backend as k
import os

#parametri de instruire
batch_size = 64
epochs =15

#incarcare baza de date 
(x_train, y_train), (x_test, y_test) = mnist.load_data()

#Stocarea numerelor in randuri si coloane
img_rows = x_train[0].shape[0]
img_cols = x_train[1].shape[0]

#Transforma datele noaste intro forma/shape  care ii este neceasa lui keras
#trebuie sa adugam 4 dimensiun pentru datale noaste prin schimbarea formei/shape
# imagini originale de la (60000,28,28) catre (60000,28,28,1)
x_train = x_train.reshape(x_train.shape[0],img_rows, img_cols,1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

#Stocheaza forma/shape a unei singure imagini
input_shape = (img_rows, img_cols,1)

#schimba typul de imagine catre float32
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#Normalizarea datelor prin schimbarea range de la (0-255) la (0-1)
x_train /=255
x_test /=255

#hot encode output
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

#Printeaza numarulde coloane in  hot encode
print("Number of Classes: " +str(y_test.shape[1]))

num_classes = y_test.shape[1]
num_pixels = x_train.shape[1] * x_train.shape[2]

#Crearea modelului
model = Sequential()

model.add(Conv2D(32,(3,3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

#compile
model.compile(keras.optimizers.Adadelta(), 'categorical_crossentropy', ['accuracy'])
print(model.summary())

checkpoint = ModelCheckpoint("c:/Users/Harum/Documents/12/MNIST_Checkpoint.h5",
                             monitor='val_loss',
                             mode='min',
                             save_best_only= True,
                             verbose=1

)
callbacks = [checkpoint]

Number of Classes: 10
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_8 (Dropout)          (No

In [8]:
history = model.fit(x_train, y_train,
                   batch_size,
                   epochs,
                   callbacks,
                   validation_data=(x_test, y_test) 
            )

score = model.evaluate(x_test, y_test, batch_size,0)
print('Test loss', score[0])
print("Test accuracy", score[1])

Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/15
Epoch 2/15
Epoch 3/15
Epoch 4/15
Epoch 5/15
Epoch 6/15
Epoch 7/15
Epoch 8/15
Epoch 9/15
Epoch 10/15
Epoch 11/15
Epoch 12/15
Epoch 13/15
Epoch 14/15
Epoch 15/15
Test loss 0.029990953985038322
Test accuracy 0.9915000200271606


# Adauga multiple Call Backs si Early Stopping

putem folosi ate metode de retur pentru a monitoriza procesul de instruire, cum ar fi Early Stopping. Consultati documentatia Keras pentru mai multe informati:
        https://keras.io/callbacks/


In [9]:
from keras.callbacks import EarlyStopping

earlystop = EarlyStopping(monitor='val_loss',        #valoarea monitorizata pentru imbunatatire
                          min_delta= 0,              # valoarea abs si modificarea minma  necesara inainte de a ne opri
                          patience=3,                #Numarul de Epochs pe care le asteptam inainte de oprie
                          verbose=1,
                          restore_best_weights=True  #pastreaza cele mai bune weigths odata oprit          
)

#pune call back intro lista callback
callbacks = [earlystop, checkpoint]

In [10]:
history = model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=3,
                    verbose=2,
                    callbacks=callbacks,
                    validation_data=(x_test, y_test)
)

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss: ', score[0])
print('Test accuracy: ', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/3
 - 13s - loss: 0.0266 - accuracy: 0.9920 - val_loss: 0.0267 - val_accuracy: 0.9917

Epoch 00001: val_loss improved from inf to 0.02672, saving model to c:/Users/Harum/Documents/12/MNIST_Checkpoint.h5
Epoch 2/3
 - 13s - loss: 0.0270 - accuracy: 0.9923 - val_loss: 0.0308 - val_accuracy: 0.9916

Epoch 00002: val_loss did not improve from 0.02672
Epoch 3/3
 - 13s - loss: 0.0268 - accuracy: 0.9918 - val_loss: 0.0355 - val_accuracy: 0.9907

Epoch 00003: val_loss did not improve from 0.02672
Test loss:  0.03554967173404625
Test accuracy:  0.9907000064849854


# O alta callback folositor este Reducerea ratei de invatare in Plateau

Putem evita sa oscilam in jurul valori minime globale prin incercarea de a reduce rata de invatare printrun anumit fact. Daca nu se observa nicio imbunatatire in matricea monitorizata (val_loss, de obicei), asteptam un anumit numar de epochs (rabdare), atunci acet callback reduce rata de invatare cu un factor.




In [11]:
from keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau('val_loss', 0.2,3,1,min_delta=0.0001)